### Recovering data from SQL SB

In [92]:
import pandas as pd
import psycopg2
import matplotlib.pyplot as plt
from math import cos, radians

pd.options.display.max_columns = 999
%matplotlib inline

In [2]:
conn = psycopg2.connect("dbname = soaring_predictor")

### Loading flying metrics

In [3]:
F = pd.read_sql('SELECT * from flight_metrics;', conn)

In [21]:
F.columns

Index(['flight', 'date', 'pilot', 'launch_time', 'landing_time', 'flying_time',
       'launch_coord', 'landing_coord', 'landed_lz', 'linear_dist', 'max_alt',
       'total_alt_gain', 'left_perimiter', 'month', 'year', 'alt_sampe',
       'len_sampe', 'doy', 'timestamp'],
      dtype='object')

In [114]:
# Isolating the best flight of the date (by max flying time)

Y = pd.DataFrame(F.groupby('date')['flying_time','max_alt','left_perimiter'].mean())

### Loading weather data

In [110]:
W = pd.read_sql('SELECT * from weather;', conn)

In [78]:
W.head()

,date,alti_0,alti_4,alti_8,alti_12,alti_16,alti_20,drct_0,drct_4,drct_8,drct_12,drct_16,drct_20,dwpf_0,dwpf_4,dwpf_8,dwpf_12,dwpf_16,dwpf_20,p01i_0,p01i_4,p01i_8,p01i_12,p01i_16,p01i_20,relh_0,relh_4,relh_8,relh_12,relh_16,relh_20,sknt_0,sknt_4,sknt_8,sknt_12,sknt_16,sknt_20,skyc1_0,skyc1_4,skyc1_8,skyc1_12,skyc1_16,skyc1_20,tmpf_0,tmpf_4,tmpf_8,tmpf_12,tmpf_16,tmpf_20,doy
0,2013-01-02,30.334000,30.2975,30.307500,30.255,30.2400,30.205000,0.000000,0.0,72.500000,150.0,87.5,55.000000,27.896,27.005,27.950000,22.505,20.210,20.255,0.00,0.000,0.0,0.0,0.0000,0.000000,93.774000,93.2300,78.742500,40.0875,45.6400,56.530000,0.000000,0.00,2.000000,5.25,3.25,2.750000,0.150000,0.000,0.000000,0.0000,0.0000,0.000000,29.480,28.715,34.520000,45.500,39.470,34.475,2
1,2013-01-03,30.195000,30.1475,30.140000,30.115,30.1250,30.152500,70.000000,35.0,120.000000,145.0,92.5,30.000000,21.515,24.305,20.525000,19.490,25.205,33.530,0.00,0.000,0.0,0.0,0.0125,0.015000,65.962500,88.6125,47.902500,36.1925,58.6150,86.257500,2.000000,1.00,4.000000,7.00,4.75,1.000000,0.000000,0.000,0.000000,0.0625,0.7500,1.000000,32.225,27.230,38.750000,44.735,39.515,37.265,3
2,2013-01-04,30.208333,30.2580,30.312857,30.270,30.2125,30.180000,93.333333,138.0,110.000000,62.0,0.0,77.500000,35.690,37.634,41.154286,43.736,41.270,38.480,0.01,0.003,0.0,0.0,0.0000,0.000000,93.811667,93.3340,91.638571,83.8080,92.7875,88.002500,2.833333,4.80,3.571429,2.40,0.00,0.750000,0.833333,0.725,0.571429,0.5500,0.1875,0.875000,37.310,39.398,43.442857,48.416,43.250,41.810,4
3,2013-01-05,30.167500,30.1700,30.192500,30.145,30.1250,30.148889,87.500000,157.5,92.500000,150.0,142.5,72.222222,38.480,34.970,33.215000,34.745,32.495,37.860,0.00,0.000,0.0,0.0,0.0000,0.017778,91.625000,76.0925,65.592500,69.7775,66.1850,91.553333,2.750000,3.75,3.500000,5.25,4.00,2.666667,0.875000,0.875,0.937500,0.7500,0.5000,0.666667,40.730,42.260,44.060000,44.015,43.025,40.200,5
4,2013-01-06,30.166000,30.1620,30.178333,30.125,30.0825,30.022500,117.000000,56.0,88.333333,27.5,90.0,57.500000,37.256,37.832,39.380000,40.550,40.235,39.245,0.00,0.000,0.0,0.0,0.0025,0.007500,94.687000,92.6860,90.475000,83.8625,85.7975,91.655000,5.300000,1.40,2.333333,0.75,3.00,3.750000,0.775000,0.750,0.750000,0.8125,0.5000,0.687500,38.660,39.776,42.020000,45.140,44.240,41.495,6


In [104]:
def get_cos(direction):
    return abs(cos(radians(direction)))

In [105]:
direction = ['drct_0', 'drct_4', 'drct_8', 'drct_12', 'drct_16', 'drct_20']

for col in direction:
    W[col]=W[col].apply(get_cos)

In [106]:
W.head()

,date,alti_0,alti_4,alti_8,alti_12,alti_16,alti_20,drct_0,drct_4,drct_8,drct_12,drct_16,drct_20,dwpf_0,dwpf_4,dwpf_8,dwpf_12,dwpf_16,dwpf_20,p01i_0,p01i_4,p01i_8,p01i_12,p01i_16,p01i_20,relh_0,relh_4,relh_8,relh_12,relh_16,relh_20,sknt_0,sknt_4,sknt_8,sknt_12,sknt_16,sknt_20,skyc1_0,skyc1_4,skyc1_8,skyc1_12,skyc1_16,skyc1_20,tmpf_0,tmpf_4,tmpf_8,tmpf_12,tmpf_16,tmpf_20,doy
0,2013-01-02,30.334000,30.2975,30.307500,30.255,30.2400,30.205000,1.000000,1.000000,0.300706,0.866025,4.361939e-02,0.573576,27.896,27.005,27.950000,22.505,20.210,20.255,0.00,0.000,0.0,0.0,0.0000,0.000000,93.774000,93.2300,78.742500,40.0875,45.6400,56.530000,0.000000,0.00,2.000000,5.25,3.25,2.750000,0.150000,0.000,0.000000,0.0000,0.0000,0.000000,29.480,28.715,34.520000,45.500,39.470,34.475,2
1,2013-01-03,30.195000,30.1475,30.140000,30.115,30.1250,30.152500,0.342020,0.819152,0.500000,0.819152,4.361939e-02,0.866025,21.515,24.305,20.525000,19.490,25.205,33.530,0.00,0.000,0.0,0.0,0.0125,0.015000,65.962500,88.6125,47.902500,36.1925,58.6150,86.257500,2.000000,1.00,4.000000,7.00,4.75,1.000000,0.000000,0.000,0.000000,0.0625,0.7500,1.000000,32.225,27.230,38.750000,44.735,39.515,37.265,3
2,2013-01-04,30.208333,30.2580,30.312857,30.270,30.2125,30.180000,0.058145,0.743145,0.342020,0.469472,1.000000e+00,0.216440,35.690,37.634,41.154286,43.736,41.270,38.480,0.01,0.003,0.0,0.0,0.0000,0.000000,93.811667,93.3340,91.638571,83.8080,92.7875,88.002500,2.833333,4.80,3.571429,2.40,0.00,0.750000,0.833333,0.725,0.571429,0.5500,0.1875,0.875000,37.310,39.398,43.442857,48.416,43.250,41.810,4
3,2013-01-05,30.167500,30.1700,30.192500,30.145,30.1250,30.148889,0.043619,0.923880,0.043619,0.866025,7.933533e-01,0.305326,38.480,34.970,33.215000,34.745,32.495,37.860,0.00,0.000,0.0,0.0,0.0000,0.017778,91.625000,76.0925,65.592500,69.7775,66.1850,91.553333,2.750000,3.75,3.500000,5.25,4.00,2.666667,0.875000,0.875,0.937500,0.7500,0.5000,0.666667,40.730,42.260,44.060000,44.015,43.025,40.200,5
4,2013-01-06,30.166000,30.1620,30.178333,30.125,30.0825,30.022500,0.453990,0.559193,0.029085,0.887011,6.123234e-17,0.537300,37.256,37.832,39.380000,40.550,40.235,39.245,0.00,0.000,0.0,0.0,0.0025,0.007500,94.687000,92.6860,90.475000,83.8625,85.7975,91.655000,5.300000,1.40,2.333333,0.75,3.00,3.750000,0.775000,0.750,0.750000,0.8125,0.5000,0.687500,38.660,39.776,42.020000,45.140,44.240,41.495,6


### Merging the 2 sources of data

In [115]:
data = pd.merge(Y,W,how='left', right_on='date',left_index=True)

In [116]:
data.dropna(inplace=True)

In [27]:
import io

from sqlalchemy import create_engine

# fail, append or replace

def write_to_table(df, db_engine, table_name, if_exists='fail'):
    string_data_io = io.StringIO()
    df.to_csv(string_data_io, sep='|', index=False)
    pd_sql_engine = pd.io.sql.pandasSQL_builder(db_engine)
    table = pd.io.sql.SQLTable(table_name, pd_sql_engine, frame=df,
                               index=False, if_exists=if_exists)
    table.create()
    string_data_io.seek(0)
    string_data_io.readline()  # remove header
    with db_engine.connect() as connection:
        with connection.connection.cursor() as cursor:
            copy_cmd = "COPY %s FROM STDIN HEADER DELIMITER '|' CSV" % table_name
            cursor.copy_expert(copy_cmd, string_data_io)
        connection.connection.commit()

In [28]:
address = 'postgresql://@localhost:5432/soaring_predictor'
engine = create_engine(address)

In [29]:
write_to_table(data, engine, 'data',if_exists='replace')

In [70]:
data.columns

Index(['flying_time', 'max_alt', 'left_perimiter', 'date', 'alti_0', 'alti_4',
       'alti_8', 'alti_12', 'alti_16', 'alti_20', 'drct_0', 'drct_4', 'drct_8',
       'drct_12', 'drct_16', 'drct_20', 'dwpf_0', 'dwpf_4', 'dwpf_8',
       'dwpf_12', 'dwpf_16', 'dwpf_20', 'p01i_0', 'p01i_4', 'p01i_8',
       'p01i_12', 'p01i_16', 'p01i_20', 'relh_0', 'relh_4', 'relh_8',
       'relh_12', 'relh_16', 'relh_20', 'sknt_0', 'sknt_4', 'sknt_8',
       'sknt_12', 'sknt_16', 'sknt_20', 'skyc1_0', 'skyc1_4', 'skyc1_8',
       'skyc1_12', 'skyc1_16', 'skyc1_20', 'tmpf_0', 'tmpf_4', 'tmpf_8',
       'tmpf_12', 'tmpf_16', 'tmpf_20', 'doy'],
      dtype='object')

In [117]:
data.to_csv('data.csv')

In [72]:
pwd

'/Users/eduardodeangelis/Desktop/galvanize/soaring-predictor/notebooks'

In [33]:
T = pd.DataFrame(W.groupby('date')['tmpf_12'].mean())

In [45]:
# T.reset_index(inplace=True)
T.drop('level_0',inplace=True,axis=1)

In [47]:
W = pd.merge(Y,T,how='left',right_on='date',left_index=True)

In [49]:
W.dropna(inplace=True)

In [55]:
# returns day of the year

def doy(dt_obj):
    return int(dt_obj.strftime('%j'))

In [57]:
W['doy']=W['date'].apply(doy)

In [65]:
# fig,ax = plt.subplots(figsize = (18,8))
# ax.scatter(W['doy'], W['flying_time'], alpha = 0.5,)
# ax.plot(W['doy'],W['tmpf_12'])
# ax.set_title('Flight time [in minutes]',fontsize=20)
# ax.set_ylabel('minutes', fontsize=16)
# ax.set_xticks([0,90,180,270]) # choose which x locations to have ticks
# ax.set_xticklabels(['Jan','Apr',"Jul",'Oct'], fontsize=14)

In [64]:
W.sort_values('doy')
# W.groupby('doy')['tmpf_12'].mean()

,flying_time,max_alt,left_perimiter,date,tmpf_12,doy
366,16.583333,500,0,2014-01-03,45.005000,3
367,16.266667,546,0,2014-01-04,43.295000,4
9,21.366667,493,0,2013-01-11,34.790000,11
10,23.066667,682,0,2013-01-12,36.005000,12
740,7.166667,563,0,2015-01-12,50.990000,12
14,16.616667,478,0,2013-01-16,39.020000,16
748,20.250000,563,0,2015-01-20,48.470000,20
384,8.150000,517,0,2014-01-21,44.735000,21
1482,61.500000,625,1,2017-01-23,45.903269,23
386,15.333333,554,0,2014-01-23,46.715000,23
